# NYC Taxi data

## Import cuxfilter

In [ ]:
import cuxfilter
from bokeh import palettes
from cuxfilter.layouts import feature_and_double_base

import cudf

In [ ]:
# update data_dir if you have downloaded datasets elsewhere
DATA_DIR = "./data"

## Download required datasets

In [ ]:
! curl https://data.rapids.ai/viz-data/nyc_taxi.tar.gz --create-dirs -o $DATA_DIR/nyc_taxi.tar.gz

In [ ]:
from cuxfilter.sampledata import datasets_check

datasets_check("nyc_taxi", base_dir=DATA_DIR)

## preprocess the data

In [ ]:
cudf_df = cudf.read_csv(DATA_DIR + "/nyc_taxi.csv").dropna()

from pyproj import Proj, Transformer

# Apply transformation
transform_4326_to_3857 = Transformer.from_crs("epsg:4326", "epsg:3857")
cudf_df["dropoff_x"], cudf_df["dropoff_y"] = transform_4326_to_3857.transform(
    cudf_df["dropoff_latitude"].values_host,
    cudf_df["dropoff_longitude"].values_host,
)

cudf_df = cudf_df.drop(["dropoff_latitude", "dropoff_longitude"], axis=1)
cudf_df = cudf_df.dropna(axis=0)

# Filter over Manhattan
cudf_df = cudf_df[
    (cudf_df.dropoff_x > -8239910.23)
    & (cudf_df.dropoff_x < -8229529.24)
    & (cudf_df.dropoff_y > 4968481.34)
    & (cudf_df.dropoff_y < 4983152.92)
]

cudf_df.tpep_pickup_datetime = cudf.to_datetime(
    cudf_df.tpep_pickup_datetime, format="%Y-%m-%d"
)
cudf_df.tpep_dropoff_datetime = cudf.to_datetime(
    cudf_df.tpep_dropoff_datetime, format="%Y-%m-%d"
)
cudf_df.head()

## Read the dataset

In [ ]:
cux_df = cuxfilter.DataFrame.from_dataframe(cudf_df)

## Define charts

In [ ]:
chart1 = cuxfilter.charts.scatter(
    x="dropoff_x",
    y="dropoff_y",
    aggregate_fn="mean",
    aggregate_col="payment_type",
    pixel_shade_type="log",
    legend_position="top_right",
    unselected_alpha=0.2,
    tile_provider="CartoDark",
    x_range=(-8239910.23, -8229529.24),
    y_range=(4968481.34, 4983152.92),
)

chart2 = cuxfilter.charts.bar("passenger_count", data_points=9)
chart3 = cuxfilter.charts.bar("tpep_pickup_datetime", xrotation=45)
chart4 = cuxfilter.charts.date_range_slider("tpep_dropoff_datetime")

## Create a dashboard object

In [ ]:
d = cux_df.dashboard(
    [chart1, chart2, chart3],
    sidebar=[chart4],
    layout=feature_and_double_base,
    theme=cuxfilter.themes.dark,
    title="NYC TAXI DATASET",
)

## Starting the dashboard

1. d.show('current_notebook_url:current_notebook_port') remote dashboard

2. d.app() inline within the notebook cell

Incase you need to stop the server:

- d.stop()

In [ ]:
# d.show() # notebook_url="http://localhost:8888"
# d.app() # run the dashboard within the notebook cell

![nyc-taxi](../docs/_images/nyc-taxi.png)

## Export the queried data into a dataframe

In [ ]:
queried_df = d.export()